In [ ]:
import numpy as np
np.set_printoptions(precision=3)

import scipy as sp
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 13})

import sys
sys.path.append("..")
import solax as sx

In [ ]:
def build_bath(N_bath):
    ii = np.arange(N_bath) + 1
    xx = ii * np.pi / (N_bath + 1)
    e_bath = -2 * np.cos(xx)
    
    V0 = np.sqrt(20 / (N_bath + 1))
    V_bath = V0 * np.sqrt(1 - (e_bath / 2)**2)
    
    return e_bath, V_bath

In [ ]:
e_bath, V_bath = build_bath(N_bath=21)

fig, axes = plt.subplots(2, 1, sharex=True, figsize=(5, 4))

ii = np.arange(1, len(e_bath) + 1)

ax = axes[0]
ax.scatter(ii, e_bath)
ax.set_ylabel(r"$\varepsilon_b$");
ax.set_ylim(-2.2, 2.2)
ax.set_yticks(np.arange(-2.0, 2.1))

ax = axes[1]
ax.scatter(ii, V_bath)
ax.set_ylabel(r"$V_b$");
ax.set_yticks(np.arange(0, 1.1, 0.25))
ax.set_ylim(-0.1, 1.1)

ax.set_xlabel("Bath site");
ax.set_xticks(ii[::2]);

In [ ]:
def build_start_dets(N_bath):
    det1 = "01"  + "1" * (N_bath - 1) + "10" + "0" * (N_bath - 1)
    det2 = "10"  + "1" * (N_bath - 1) + "01" + "0" * (N_bath - 1)
    return det1, det2

In [ ]:
U = 10
N_bath = 3
e_bath, V_bath = build_bath(N_bath)
start_dets = build_start_dets(N_bath)

In [ ]:
basis_start = sx.Basis(build_start_dets(N_bath))
print(basis_start)

In [ ]:
H_imp2 = sx.Operator(
    (1, 0, 1, 0),
    np.array([
        [0, 0, 1, 1]
    ]),
    np.array([U])
)

H_imp1 = sx.Operator(
    (1, 0),
    np.array([
        [0, 0],
        [1, 1]
    ]),
    np.array([-U / 2, -U / 2])
)

H_imp = H_imp2 + H_imp1 + U / 4
print(H_imp)

In [ ]:
H_bath = sx.Operator(
    (1, 0),
    np.arange(2, 2 * N_bath + 2).repeat(2).reshape(-1, 2),
    e_bath.repeat(2)
)
print(H_bath)

In [ ]:
H_hyb_posits = np.vstack([
    np.array([0, 1] * N_bath),
    np.arange(2, 2 * N_bath + 2)
]).T

H_hyb_nohc = sx.Operator(
    (1, 0),
    H_hyb_posits,
    V_bath.repeat(2)
)
print(H_hyb_nohc)

In [ ]:
H = H_imp + H_bath + H_hyb_nohc + H_hyb_nohc.hconj

In [ ]:
matrix_start = H.build_matrix(basis_start)

In [ ]:
matrix_dense_start = matrix_start.to_scipy().todense()
print(matrix_dense_start)

In [ ]:
energy_start = matrix_dense_start[0, 0]
print(energy_start)

In [ ]:
basis = H(basis_start)
print(len(basis))

In [ ]:
matrix = H.build_matrix(basis)

matrix_dense = matrix.to_scipy().todense()
print(matrix_dense)

In [ ]:
energy = np.linalg.eigvals(matrix_dense).min()

basis_size = len(basis)
print(f"Basis size = {basis_size}\tEnergy = {energy}")

In [ ]:
U = 10
N_bath = 21
e_bath, V_bath = build_bath(N_bath)
start_dets = build_start_dets(N_bath)

basis_start = sx.Basis(build_start_dets(N_bath))

H_imp2 = sx.Operator(
    (1, 0, 1, 0),
    np.array([
        [0, 0, 1, 1]
    ]),
    np.array([U])
)

H_imp1 = sx.Operator(
    (1, 0),
    np.array([
        [0, 0],
        [1, 1]
    ]),
    np.array([-U / 2, -U / 2])
)

H_imp = H_imp2 + H_imp1 + U / 4

H_bath = sx.Operator(
    (1, 0),
    np.arange(2, 2 * N_bath + 2).repeat(2).reshape(-1, 2),
    e_bath.repeat(2)
)

H_hyb_posits = np.vstack([
    np.array([0, 1] * N_bath),
    np.arange(2, 2 * N_bath + 2)
]).T

H_hyb_nohc = sx.Operator(
    (1, 0),
    H_hyb_posits,
    V_bath.repeat(2)
)

H = H_imp + H_bath + H_hyb_nohc + H_hyb_nohc.hconj

In [ ]:
num_iterations = 4

basis = basis_start

for i in range(num_iterations):
    matrix = H.build_matrix(basis)
    energy = sp.sparse.linalg.eigsh(
        matrix.to_scipy(), k=1, which="SA"
    )[0][0]
    
    basis_size = len(basis)
    print(
        f"Iteration: {i+1:<8d}"
        f"Basis size = {basis_size:<12d}"
        f"Energy = {energy}"
    )
    
    if i < num_iterations - 1:
        basis = H(basis)

In [ ]:
basis_small = basis
M_small = matrix
print(M_small.size)

In [ ]:
basis_big = H(basis_small)
M_big_direct = H.build_matrix(basis_big)
print(M_big_direct.size)

In [ ]:
print(len(basis_small % basis_big) == 0)

In [ ]:
basis_cols = basis_big % basis_small
basis_rows = basis_small + basis_cols

In [ ]:
C = H.build_matrix(basis_rows, basis_cols)
print(C.size)

In [ ]:
C_displ = C.displace(0, len(basis_small))

In [ ]:
M_with2B = M_small + C_displ + C_displ.hconj

In [ ]:
left_top = (len(basis_small), len(basis_small))
right_bottom = (None, None)

B_displ = C_displ.window(left_top, right_bottom)

In [ ]:
M_big = M_with2B - B_displ

In [ ]:
energy_big_direct = sp.sparse.linalg.eigsh(M_big_direct.to_scipy(), k=1, which="SA")[0][0]
print(energy_big_direct)

In [ ]:
energy_big = sp.sparse.linalg.eigsh(M_big.to_scipy(), k=1, which="SA")[0][0]
print(energy_big)